#Student Name: Vipul Popat
#Student ID:   18195121

***Task***
With the final version of the code from the peer you selected in E-tivity 3 as a starting point, create a class that contains one tensor and provides the following methods:

I have used the final version of Etivity Code of Brian Parle (Version ad6233c7390ebe909565f03c0594f5fb56213ec3) in order to get this class done. 

- Calculate the size of the tensor as a 2D tuple
- Sum/subtract the tensor with another tensor of suitable size
- Multiply the tensor with a tensor of suitable size
- Calculate the determinant of the tensor, but only for tensors of size 2x2 (matrix)
- Calculate the inverse of the tensor, but only for tensors of size 2x2
- Calculate the cross product of the tensor with another tensor of suitable size
- Have done exception handling in terms of TypeError and ValueError at various places where the expected types and values are not passed as required.
- Renamed the class to Tensor to fall in line with the expectations of the Etivity.

Thanks to 
- Brian Parle (for his Initial Code and detailed comments and review on the code)
- Abhijit Sinha for pointing out the BUG in matrix multiplication and insights
- Chells for pointing out the unused variables mentioned when they would be anyways instantiated by the constructor
- Mark Murnane for his insight into overloading of \_\_rmul\_\_  and exception handling



In [6]:
class Tensor:
    """ Class Representing a Tensor
        This class provides all the mathematical operations between scalars and 
        other Tensors of expected ranks"""
    # instance variables
    dimension_2x2 = (2,2)
    dimension_3x1 = (3,1)
    
    def __init__(self, m_tuples):
        self.m_tuples = m_tuples
        self.dimension = self.__tensor_get_size__()

    def __add__(self, other_tensor):
        return self.__tensor_add_or_subtract__(other_tensor, Tensor.fn_add)

    def __sub__(self, other_tensor):
        return self.__tensor_add_or_subtract__(other_tensor, Tensor.fn_subtract)

    def __mul__(self, other_tensor):
        if isinstance(other_tensor, (int, float)):
            return self.__scalar_multiplication__(other_tensor)
        elif isinstance(other_tensor, self.__class__):
            return self.__tensor_multiplication__(other_tensor)
        else:
            return TypeError("The tensor multiplication operator can only be applied to another Tensor object or scalars")            

    def __rmul__(self, other):
        return self.__scalar_multiplication__(other)
    
    def __tensor_get_size__(self):
        """
        Tensor size is (number of rows, number of columns)
        """
        return (len(self.m_tuples), len(self.m_tuples[0]))    
    
    #had to make this method static in order to reuse this in the functions
    @staticmethod
    def fn_add(x,y):
        return x + y
    
    #had to make this method static in order to reuse this in the functions
    @staticmethod
    def fn_subtract(x,y):
        return x - y    
    
    def __scalar_multiplication__(self, other):
        """
        Purpose: multiple a tensor with a scalar
        """
        result = []
        for tup in self.m_tuples:
            result.append(tuple([x*other for x in tup]))
        return Tensor(tuple(result))
            

    def __tensor_multiplication__(self, other):
        """
        Purpose: multiple 2 two compatible tensors
        """
        if not self.__compatible_tensor_size__(other, "multiply"):
            return []
        
        m1_rowcount = len(self.m_tuples)
        m1_colcount = len(self.m_tuples[0])
        m2_colcount = len(other.m_tuples[0])
        new_tensor = []
        for m1_rowindex in range(m1_rowcount):
            #for this row in tensor1, multiple each rowitem by each item in the corresponding column in tensor2
            new_row = []
            for m2_colindex in range(m2_colcount):
                #for this column in tensor2, sum the product of each row item with each column item
                cell_sum = 0
                for m1_colindex in range(m1_colcount):
                    #iterate through each item in the tensor1 row
                    cell_sum += self.m_tuples[m1_rowindex][m1_colindex] * other.m_tuples[m1_colindex][m2_colindex]
                #add this item to the new tensor row
                new_row.append (cell_sum)
            #add the new tensor row to the output tensor
            new_tensor.append (tuple(new_row))
        return Tensor(tuple(new_tensor))
    
    def __tensor_add_or_subtract__(self, other, fn_operate):
        """
        Purpose: Add or subtract 2 tensors. Add or subtract function is passed as argument
        """
        if not self.__compatible_tensor_size__( other, "add_or_subtract"):
            return []
    
        new_tensor = []
        for rowindex, row in enumerate(self.m_tuples):
            new_row = []
            for colindex, col in enumerate(self.m_tuples[rowindex]):
                new_row.append (fn_operate(row[colindex] , other.m_tuples[rowindex][colindex]))
            new_tensor.append (tuple(new_row))
        return tuple(new_tensor)

    def __compatible_tensor_size__(self, other, op):
        if op == "multiply":
            # Corrected on comments by Abhijit Sinha.
            if self.dimension[1] != other.dimension[0]:
                #tensor 1 does not have the same number of columns as tensor 2, so incompatible
                return False
        elif op == "add_or_subtract":
            if self.dimension != other.dimension:
                #tensors are of different dimensions, so incompatible
                return False
        #no errors, so compatible
        return True

    def __matmul__(self, other):
        self.__mul__(other)

    def __str__(self):
        return str(self.m_tuples)
    
    def __eq__(self, other_tensor):
        return self.m_tuples == other_tensor.m_tuples
    
    def determinant(self):
        """This function calculates the determinant of a 2x2 tensor"""
        if self.dimension != Tensor.dimension_2x2:
            raise ValueError("This function is only for 2x2 tensor")
        return self.m_tuples[0][0] * self.m_tuples[1][1] - self.m_tuples[0][1] * self.m_tuples[1][0]

    def inverse(self):
        """This function calculates the Inverse of a 2x2 tensor"""
        if self.dimension != Tensor.dimension_2x2:
            raise ValueError("This function is only for 2x2 tensor")
        if self.determinant() == 0:
            raise ValueError("This Tensor is not compatible for inverse, the determinant is 0")
            
        multiplier = 1 / self.determinant()
        new_m_tuple = ((self.m_tuples[1][1],self.m_tuples[0][1] * -1),(self.m_tuples[1][0] * -1, self.m_tuples[0][0]))
        return Tensor(new_m_tuple) * multiplier
    
    def cross_product(self, other):
        """This function calculates the Cross product of two 3x1 vectors"""
        # Calculate the cross product of the tensor with another tensor of suitable size
        #This implementation only works with vectors with rank 3, other rank vectors other than 7 do not have a cross product
        if self.dimension != Tensor.dimension_3x1 and other.dimension != Tensor.dimension_3x1:
            raise ValueError("Cross product is only implemented 3d vectors")

        v1 = self.m_tuples
        v2 = other.m_tuples
        m1 = Tensor(((v1[1][0], v2[1][0]),(v1[2][0], v2[2][0])))
        m2 = Tensor(((v1[2][0], v2[2][0]),(v1[0][0], v2[0][0])))
        m3 = Tensor(((v1[0][0], v2[0][0]),(v1[1][0], v2[1][0])))
        return Tensor(((m1.determinant(),),(m2.determinant(),),(m3.determinant(),)))
    
    
# Test tensors
tensor_1 = Tensor(((7, 4, -3, 2), (-1, 0, 6, 1), (3, 9, 2, 5), (8, 5, 9, -6)))
tensor_2 = Tensor(((-4, 9, -7, 4), (-3, 6, 0, 8), (0, -5, -2, 1), (2, 1, 3, -8)))
tensor_3 = Tensor(((5,), (-4,), (3,), (-2,)))

# Sum, Subtract and Multiply 4x4 Matrices
print("tensor1 + tensor_2 = ", tensor_1 + tensor_2)
print("tensor1 - tensor_2 = ", tensor_1 - tensor_2)
print("tensor1 * tensor_2 = ", tensor_1 * tensor_2)
print("tensor2 * tensor_1 = ", tensor_2 * tensor_1)

# Scalar multiplications
print("tensor2 * 3 = ", tensor_2 * 3)
print("3 * tensor2 = ", 3 * tensor_2)

# Multiply Tensor by Vector and Vector by Tensor
print("tensor2 * tensor_3 = ", tensor_2 * tensor_3)

tensor_2x2_m1 = ((2,4),(3,-1))
# Print determinant and inverse of a tensor
print("Determinant of tensor_2x2_m1 = ", Tensor(tensor_2x2_m1).determinant())
print("Inverse of tensor_2x2_m1 = ", Tensor(tensor_2x2_m1).inverse())

#Cross product
tensor_3x1_1=Tensor(((1,),(2,),(-1,)))
tensor_3x1_2=Tensor(((-4,),(1,),(-2,)))
print("Cross product of tensor_3x1_1 x tensor_3x1_2 = ", tensor_3x1_1.cross_product(tensor_3x1_2))
# printing the identity tensor
print ("tensor_2_2_m1 * its' inverse = I: ", Tensor(tensor_2x2_m1).inverse() * Tensor(tensor_2x2_m1))

print(Tensor.__doc__)

tensor1 + tensor_2 =  ((3, 13, -10, 6), (-4, 6, 6, 9), (3, 4, 0, 6), (10, 6, 12, -14))
tensor1 - tensor_2 =  ((11, -5, 4, -2), (2, -6, 6, -7), (3, 14, 4, 4), (6, 4, 6, 2))
tensor1 * tensor_2 =  ((-36, 104, -37, 41), (6, -38, -2, -6), (-29, 76, -10, 46), (-59, 51, -92, 129))
tensor2 * tensor_1 =  ((-26, -59, 88, -58), (37, 28, 117, -48), (7, -13, -25, -21), (-42, -5, -66, 68))
tensor2 * 3 =  ((-12, 27, -21, 12), (-9, 18, 0, 24), (0, -15, -6, 3), (6, 3, 9, -24))
3 * tensor2 =  ((-12, 27, -21, 12), (-9, 18, 0, 24), (0, -15, -6, 3), (6, 3, 9, -24))
tensor2 * tensor_3 =  ((-85,), (-55,), (12,), (31,))
Determinant of tensor_2x2_m1 =  -14
Inverse of tensor_2x2_m1 =  ((0.07142857142857142, 0.2857142857142857), (0.21428571428571427, -0.14285714285714285))
Cross product of tensor_3x1_1 x tensor_3x1_2 =  ((-3,), (6,), (9,))
tensor_2_2_m1 * its' inverse = I:  ((1.0, 0.0), (0.0, 1.0))
 Class Representing a Tensor
        This class provides all the mathematical operations between scalars and 
     